In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from matrix_factorization import matrix_factorization
from graph_init import *
from similarity import *
from create_R import *
from ALS import *
from hard_hfs import *
import copy

In [2]:
def RMSE(ground, predict):
    
    error = 0
    n = 0
    
    for i in range(len(ground)):
        for j in range(len(ground[0])):
            if ground[i,j] != 0:
                error += (ground[i,j] - predict[i,j])**2
                n += 1
                
    return np.sqrt(error/n)

In [3]:
# R = [
#      [5,3,0,1],
#      [4,0,0,1],
#      [1,1,0,5],
#      [1,0,0,4],
#      [0,1,5,4],
#     ]

# R = np.array(R)

R,R_dict = create_R()

print(R_dict)

/home/marc/Documents/MVA/ProjetGraphes/src/create_R.py:21: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  R[ratingsnp[i,0]-1, ratingsnp[i,-1]] = ratingsnp[i,2]


{'Users': array([   0.,    0.,    0., ...,  670.,  670.,  670.]), 'Movies': array([   30.,   833.,   859., ...,  4597.,  4610.,  4696.]), 'Ratings': array([ 2.5,  3. ,  3. , ...,  4. ,  2.5,  3.5])}


In [4]:
P_dict = copy.deepcopy(R_dict)
P_dict["Ratings"] = np.ones([len(R_dict["Ratings"])])
P = R > 0
print(P)

[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., False False False]
 [ True False False ..., False False False]
 [ True False False ..., False False False]]


In [5]:
N = len(R)
M = len(R[0])
K = 4

# P = np.random.rand(N,K)
# Q = np.random.rand(M,K)

# nP, nQ = matrix_factorization(R, P, Q, K)

als = ALS(K,N,M,"Users","Movies","Ratings",lbda = 0.1,lbda2 = 0.1)
print("Als created")
ans = als.fitImplicit(P_dict, alpha = 1, c = "log")

# nR = np.dot(nP, nQ.T)

# print(nP, "\n\n", nQ)

Als created


In [6]:
als.U

array([[ 0.07681321, -0.13825379,  0.05134058, -0.04214472],
       [ 0.1590468 , -0.30002444,  0.20382886, -0.22771697],
       [ 0.01442915, -0.28866064,  0.07008824, -0.18833544],
       ..., 
       [ 0.11919088, -0.19569136,  0.00074576, -0.04450658],
       [ 0.04358653, -0.23326602,  0.12605617, -0.17099388],
       [-0.01560695, -0.39175547,  0.17299209, -0.22690621]])

In [7]:
als.V

array([[ 0.30602635, -2.14325065,  0.3590047 , -0.65632635],
       [ 0.57560618, -1.85319394, -0.09092429, -0.48664782],
       [ 1.10823851, -1.43895913,  0.03879683, -0.21305967],
       ..., 
       [-0.0715317 , -0.03614584, -0.06034751, -0.1229135 ],
       [-0.0715317 , -0.03614584, -0.06034751, -0.1229135 ],
       [-0.33336911, -0.20255162,  0.24005682,  0.53362769]])

In [8]:
R_rec = np.dot(als.U,np.transpose(als.V))
# print(2*(R_rec-np.min(R_rec))*5/np.max(R_rec-np.min(R_rec))/2)
print(R_rec)

[[ 0.3659116   0.31626678  0.29504011 ...,  0.0015846   0.0015846
  -0.00776855]
 [ 0.91433225  0.73983676  0.66440991 ...,  0.01515667  0.01515667
  -0.06483642]
 [ 0.77185932  0.62852996  0.47420769 ...,  0.02832106  0.02832106
  -0.0300174 ]
 ..., 
 [ 0.48536977  0.45285228  0.42319529 ...,  0.00397296  0.00397296
  -0.02366787]
 [ 0.67076872  0.52912808  0.42528702 ...,  0.01872406  0.01872406
  -0.02826842]
 [ 1.04588352  0.81170963  0.60147999 ...,  0.03272692  0.03272692
   0.00499808]]


In [9]:
print(RMSE(P,R_rec))
print(RMSE(P,(R_rec-np.min(R_rec))*1/np.max(R_rec-np.min(R_rec))))

0.387185346333
0.325535421738


In [10]:
R

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 4.,  0.,  0., ...,  0.,  0.,  0.],
       [ 5.,  0.,  0., ...,  0.,  0.,  0.]])

In [11]:
np.max(R_rec)

1.5789676258706624

In [12]:
lp = LaplacianParams()

sim = similarity(als.V)

print(sim)

[[  1.00000000e+03   7.94282993e-01   7.71258084e-01 ...,   6.79686675e-01
    6.79686675e-01   6.41704554e-01]
 [  7.94282993e-01   1.00000000e+03   7.86505756e-01 ...,   6.75496157e-01
    6.75496157e-01   6.27369921e-01]
 [  7.71258084e-01   7.86505756e-01   1.00000000e+03 ...,   6.34702126e-01
    6.34702126e-01   6.18451862e-01]
 ..., 
 [  6.79686675e-01   6.75496157e-01   6.34702126e-01 ...,   1.00000000e+03
    8.01378230e-01   5.84186051e-01]
 [  6.79686675e-01   6.75496157e-01   6.34702126e-01 ...,   8.01378230e-01
    1.00000000e+03   5.84186051e-01]
 [  6.41704554e-01   6.27369921e-01   6.18451862e-01 ...,   5.84186051e-01
    5.84186051e-01   1.00000000e+03]]


In [13]:
L = build_laplacian(sim,lp)

print(L)

[[  8.06605000e+03  -7.94282993e-01  -7.71258084e-01 ...,  -6.79686675e-01
   -6.79686675e-01  -6.41704554e-01]
 [ -7.94282993e-01   8.06605000e+03  -7.86505756e-01 ...,  -6.75496157e-01
   -6.75496157e-01  -6.27369921e-01]
 [ -7.71258084e-01  -7.86505756e-01   8.06605000e+03 ...,  -6.34702126e-01
   -6.34702126e-01  -6.18451862e-01]
 ..., 
 [ -6.79686675e-01  -6.75496157e-01  -6.34702126e-01 ...,   8.06605000e+03
   -8.01378230e-01  -5.84186051e-01]
 [ -6.79686675e-01  -6.75496157e-01  -6.34702126e-01 ...,  -8.01378230e-01
    8.06605000e+03  -5.84186051e-01]
 [ -6.41704554e-01  -6.27369921e-01  -6.18451862e-01 ...,  -5.84186051e-01
   -5.84186051e-01   8.06605000e+03]]


In [ ]:
hfs0 = soft_hfs(als.V, P[0]+1, 100, 1, L)

In [67]:
hfs0

array([0, 0, 0, ..., 0, 0, 0])

In [74]:
np.max(hfs0[1])

-0.00033666922728044094

In [76]:
np.max(hfs0[0][P[0]!=0])

0.00037000066897167227